In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#Opção 
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [88]:
#Calculo do KS 
def ks(data=None,target=None, prob=None):
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 10)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2%}'.format)
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2%}'.format)
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2%}'.format)
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2%}'.format)
    kstable.index = range(1,11)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    print(kstable)
    
    #Display KS
    from colorama import Fore
    print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return(kstable)

In [81]:
##  create a dataframe
df = pd.read_csv("Base_modelagem_3.csv", sep=";")

In [82]:
df2=pd.get_dummies(df, prefix=['col1', 'col2', 'col3','col4','col5','col6', 'col7','col8','col9'])

In [83]:
df2=df2.rename({'col1_(-Inf,3]':'col1_a', 	'col1_(3, Inf]':'col1_b', 	'col1_Missing':'col1_c', 	'col2_(-Inf,Inf]':'col2_a', 	'col2_Missing':'col2_b', 	'col3_(-Inf,9]':'col3_a', 	'col3_(9, Inf]':'col3_b', 	'col3_Missing':'col3_c', 	'col4_(-Inf,2]':'col4_a', 	'col4_(2, Inf]':'col4_b', 	'col4_Missing':'col4_c', 	'col5_0':'col5_a', 	'col5_1':'col5_b', 	'col5_2':'col5_c', 	'col5_3':'col5_d', 	'col5_4':'col5_e', 	'col5_missing':'col5_f', 	'col6_1':'col6_a', 	'col6_2':'col6_b', 	'col6_3':'col6_c', 	'col6_4':'col6_d', 	'col6_missing':'col6_e', 	'col7_1':'col7_a', 	'col7_2':'col7_b', 	'col7_3':'col7_c', 	'col7_4':'col7_d', 	'col7_5':'col7_e', 	'col7_6':'col7_f', 	'col7_missing':'col7_g', 	'col8_(-Inf,0]':'col8_a', 	'col8_(0,1]':'col8_b', 	'col8_(1, Inf]]':'col8_c', 	'col8_Missing':'col8_d', 	'col9_(-Inf,3]':'col9_a', 	'col9_(3, Inf]]':'col9_b', 	'col9_Missing':'col9_c', }, axis='columns')


In [90]:
df_z= df2.loc[:,['PAGTO','col1_a', 	'col1_b', 	'col1_c', 	'col2_a', 	'col2_b', 	'col3_a']]

In [102]:
df_z= df2.loc[:,['PAGTO','col1_a', 	'col1_b', 	'col1_c', 	'col2_a', 	'col2_b', 	'col3_a', 	'col3_b', 	'col3_c', 	'col4_a', 	'col4_b', 	'col4_c', 	'col5_a', 	'col5_b', 	'col5_c', 	'col5_d', 	'col5_e', 	'col5_f', 	'col6_a', 	'col6_b', 	'col6_c', 	'col6_d', 	'col6_e', 	'col7_a', 	'col7_b', 	'col7_c', 	'col7_d', 	'col7_e', 	'col7_f', 	'col7_g', 	'col8_a', 	'col8_b', 	'col8_c', 	'col8_d', 	'col9_a', 	'col9_b', 	'col9_c']]

In [103]:
df_z= df_z.sample(200000) 

In [104]:
# Tirar a primeira columa (resposta)
colunas=df_z.columns[1:]

In [105]:
#Gera todas as combinações possíveis 
for L in range(0, len(colunas)+1):
    for subset in itertools.combinations(colunas, L):
        vare = "PAGTO ~ "
        for number in subset[0:]:
            #gera a formula 
            vare = vare + number + '+'
            #tirar o último +
            formula=vare[:vare.rfind('+')]
        #só começar a imprimir quando tiver no minimo 7 variávies
        if len(subset)>12: 
            print(subset)
            print(formula)
            print('----------------------')
        

            #gera o modelo  
            model = smf.glm(formula =formula , data=df_z, family=sm.families.Binomial())
            result = model.fit()
            print(result.summary())
            print('----------------------')
            print('----------------------')

            #scorar a base 
            #Tirar o intercepto
            betas=result.params[1:]
            len(betas)

            #score
            df_z['score']=df_z[colunas[0]]*0+result.params[0]
            for i in range(0,len(betas)):
                df_z['score']= df_z['score']+betas[i]*df_z[colunas[i]]

            df_z['score_1']=round((np.exp(df_z['score']) /(1+np.exp(df_z['score'])))*1000,0)

            ## Ks
            try:
                mydf=ks(data=df_z,target="PAGTO", prob="score_1")
                print(mydf)
            except ValueError:
                print("não foi possivel calcular o KS")

            print('----------------------')
            test=stats.ks_2samp(df_z['score_1'], df_z['PAGTO'])
            print('KS de:')
            print(test) 
            df_z=df_z.drop(['target0', 'score_1', 'score' ], axis=1)

KeyboardInterrupt: 

In [44]:
vare = "PAGTO ~ "

for number in colunas[0:]:
        #gera a formula 
        vare = vare + number + '+'
        #tirar o último +
        formula=vare[:vare.rfind('+')]
        print('----------------------')
        print(number)
        print(formula)
        

----------------------
col1_a
PAGTO ~ col1_a
----------------------
col1_b
PAGTO ~ col1_a+col1_b
----------------------
col1_c
PAGTO ~ col1_a+col1_b+col1_c
----------------------
col2_a
PAGTO ~ col1_a+col1_b+col1_c+col2_a
----------------------
col2_b
PAGTO ~ col1_a+col1_b+col1_c+col2_a+col2_b
----------------------
col3_a
PAGTO ~ col1_a+col1_b+col1_c+col2_a+col2_b+col3_a


In [36]:
vare

'PAGTO ~ +col1_a+col1_b+col1_c+col2_a+col2_b+col3_a'

43

'PAGTO ~ col1_a+col1_b+col1_c+col2_a+col2_b+col3_a'

In [21]:
from itertools import permutations

caracteres = [0, 1, 2]
for subset in permutations(caracteres, 2):
    print(subset)

(0, 1)
(0, 2)
(1, 0)
(1, 2)
(2, 0)
(2, 1)


In [45]:
import itertools

stuff = [1, 2, 3]
for L in range(0, len(stuff)+1):
    for subset in itertools.combinations(stuff, L):
        print(subset)

()
(1,)
(2,)
(3,)
(1, 2)
(1, 3)
(2, 3)
(1, 2, 3)


In [ ]:
vare = "PAGTO ~ "

for number in colunas[0:]:
        #gera a formula 
        vare = vare + number + '+'
        #tirar o último +
        formula=vare[:vare.rfind('+')]
        print('----------------------')
        print(number)
        print(formula)
        

In [78]:
colunas

Index(['col1_a', 'col1_b', 'col1_c', 'col2_a', 'col2_b', 'col3_a'], dtype='object')

In [73]:
#Gera todas as combinações possíveis 
for L in range(0, len(colunas)+1):
    for subset in itertools.combinations(colunas, L):
        print(subset)
        

()
('col1_a',)
('col1_b',)
('col1_c',)
('col2_a',)
('col2_b',)
('col3_a',)
('col1_a', 'col1_b')
('col1_a', 'col1_c')
('col1_a', 'col2_a')
('col1_a', 'col2_b')
('col1_a', 'col3_a')
('col1_b', 'col1_c')
('col1_b', 'col2_a')
('col1_b', 'col2_b')
('col1_b', 'col3_a')
('col1_c', 'col2_a')
('col1_c', 'col2_b')
('col1_c', 'col3_a')
('col2_a', 'col2_b')
('col2_a', 'col3_a')
('col2_b', 'col3_a')
('col1_a', 'col1_b', 'col1_c')
('col1_a', 'col1_b', 'col2_a')
('col1_a', 'col1_b', 'col2_b')
('col1_a', 'col1_b', 'col3_a')
('col1_a', 'col1_c', 'col2_a')
('col1_a', 'col1_c', 'col2_b')
('col1_a', 'col1_c', 'col3_a')
('col1_a', 'col2_a', 'col2_b')
('col1_a', 'col2_a', 'col3_a')
('col1_a', 'col2_b', 'col3_a')
('col1_b', 'col1_c', 'col2_a')
('col1_b', 'col1_c', 'col2_b')
('col1_b', 'col1_c', 'col3_a')
('col1_b', 'col2_a', 'col2_b')
('col1_b', 'col2_a', 'col3_a')
('col1_b', 'col2_b', 'col3_a')
('col1_c', 'col2_a', 'col2_b')
('col1_c', 'col2_a', 'col3_a')
('col1_c', 'col2_b', 'col3_a')
('col2_a', 'col2_b', 

In [79]:
#Gera todas as combinações possíveis 
for L in range(0, len(colunas)+1):
    for subset in itertools.combinations(colunas, L):
        vare = "PAGTO ~ "
        for number in subset[0:]:
            #gera a formula 
            vare = vare + number + '+'
            #tirar o último +
            formula=vare[:vare.rfind('+')]
        print(subset)
        print(formula)
        print('----------------------')

()
PAGTO ~ col1_a+col1_b+col1_c+col2_a+col2_b+col3_a
----------------------
('col1_a',)
PAGTO ~ col1_a
----------------------
('col1_b',)
PAGTO ~ col1_b
----------------------
('col1_c',)
PAGTO ~ col1_c
----------------------
('col2_a',)
PAGTO ~ col2_a
----------------------
('col2_b',)
PAGTO ~ col2_b
----------------------
('col3_a',)
PAGTO ~ col3_a
----------------------
('col1_a', 'col1_b')
PAGTO ~ col1_a+col1_b
----------------------
('col1_a', 'col1_c')
PAGTO ~ col1_a+col1_c
----------------------
('col1_a', 'col2_a')
PAGTO ~ col1_a+col2_a
----------------------
('col1_a', 'col2_b')
PAGTO ~ col1_a+col2_b
----------------------
('col1_a', 'col3_a')
PAGTO ~ col1_a+col3_a
----------------------
('col1_b', 'col1_c')
PAGTO ~ col1_b+col1_c
----------------------
('col1_b', 'col2_a')
PAGTO ~ col1_b+col2_a
----------------------
('col1_b', 'col2_b')
PAGTO ~ col1_b+col2_b
----------------------
('col1_b', 'col3_a')
PAGTO ~ col1_b+col3_a
----------------------
('col1_c', 'col2_a')
PAGTO ~ c

In [65]:
subset

('col1_a', 'col1_b', 'col1_c', 'col2_a', 'col2_b', 'col3_a')

In [ ]:
vare = "PAGTO ~ "
for number in len(subset):
        #gera a formula 
        vare = vare + number + '+'
        #tirar o último +
        formula=vare[:vare.rfind('+')]
        print('----------------------')
        print(number)
        print(formula)

In [71]:
vare = "PAGTO ~ "
for number in subset[0:]:
        #gera a formula 
        vare = vare + number + '+'
        #tirar o último +
        formula=vare[:vare.rfind('+')]
        print('----------------------')
        print(number)
        print(formula)

----------------------
col1_a
PAGTO ~ col1_a
----------------------
col1_b
PAGTO ~ col1_a+col1_b
----------------------
col1_c
PAGTO ~ col1_a+col1_b+col1_c
----------------------
col2_a
PAGTO ~ col1_a+col1_b+col1_c+col2_a
----------------------
col2_b
PAGTO ~ col1_a+col1_b+col1_c+col2_a+col2_b
----------------------
col3_a
PAGTO ~ col1_a+col1_b+col1_c+col2_a+col2_b+col3_a


In [60]:
colunas

Index(['col1_a', 'col1_b', 'col1_c', 'col2_a', 'col2_b', 'col3_a'], dtype='object')

In [59]:
for number in teste[0:]:
        #gera a formula 
        vare = vare + number + '+'
        #tirar o último +
        formula=vare[:vare.rfind('+')]
        print('----------------------')
        print(number)
        print(formula)

----------------------
col1_a
PAGTO ~ col1_acol1_a+col1_b+col1_c+col2_a+col2_b+col3_a+col1_a
----------------------
col1_b
PAGTO ~ col1_acol1_a+col1_b+col1_c+col2_a+col2_b+col3_a+col1_a+col1_b
----------------------
col1_c
PAGTO ~ col1_acol1_a+col1_b+col1_c+col2_a+col2_b+col3_a+col1_a+col1_b+col1_c
----------------------
col2_a
PAGTO ~ col1_acol1_a+col1_b+col1_c+col2_a+col2_b+col3_a+col1_a+col1_b+col1_c+col2_a
----------------------
col2_b
PAGTO ~ col1_acol1_a+col1_b+col1_c+col2_a+col2_b+col3_a+col1_a+col1_b+col1_c+col2_a+col2_b
----------------------
col3_a
PAGTO ~ col1_acol1_a+col1_b+col1_c+col2_a+col2_b+col3_a+col1_a+col1_b+col1_c+col2_a+col2_b+col3_a
